# Closest N points

Given points in a cartesian plane (x,y), n points in that plane, and a point of interest (x_1, y_1), find the n closest points to the point of interest.

Define the points in an NxD array 

## Solution

Dynamic programming

Keep a sorted solution set.  For each new candidate point, test it against the furthers kept point.  If it's smaller, insert it in order.

## Complexity

We need to touch every point once, so that's at least O(N).  
Inserting into our solution set could cost O(m) to maintain order.
In the worst case, keeping a large number of the points, in a set of points with descending distance, we'd have to insert every new point, and maintain order in our solition set.

So the solution costs O(n X m), but in most cases the m portion is insignificant compared to n, so it would be O(n)

We also need to keep our solution set in memory, but no more, so the space complexity is O(m)

In [3]:
import numpy as np

def L2_norm(a, b):
    # L2 norm
    return np.sqrt( (a[0]-b[0])**2 + (a[1]-b[1])**2 )
        
    
def find_m_closest_points(points_to_search, point_of_interest, m, dist_func = L2_norm):
    closest_points = []
    
    # collect the first closest points
    for pi_idx in range(m):
        pi = all_points_array[pi_idx]
        closest_points.append([dist_func(point_of_interest, pi), pi_idx])

    closest_points = sorted(closest_points)
    
    # test all the rest
    for idx, pi in enumerate(all_points_array[m:]):
        delta_i = dist_func(point_of_interest, pi)
        # keeps most recent for ties
        if delta_i <= closest_points[-1][0]:
            # drop further currently kept
            closest_points.pop(-1)
            new_point_tup = [delta_i, idx]
            inserted = False
            for jdx, kept_point in enumerate(closest_points):
                if delta_i <= kept_point[0]:
                    closest_points = closest_points[:jdx] + [new_point_tup] + closest_points[jdx:]
                    inserted = True
                    break
            # if we haven't placed yet, it goes at the end
            if not inserted:
                closest_points.append(new_point_tup)
    
    return closest_points



n = 1000
m = 5
points_range = 10
point_of_interest = np.array([1.0,1.0])
all_points_array = (np.random.rand(n,2) * points_range) - (points_range/2)

m_closest_points = find_m_closest_points(all_points_array, point_of_interest, m, L2_norm)

print("{0} closest points to {1} are :".format(m, point_of_interest))
print("")
for delta_i, idx in m_closest_points:
    s = "{0}, at a distance of : {1}".format(all_points_array[idx], delta_i)
    print(s)


5 closest points to [ 1.  1.] are :

[ 2.40378091 -3.86121024], at a distance of : 0.06282852869857627
[ 1.09309992  2.93149129], at a distance of : 0.31370970116522423
[-4.37000299 -1.15437599], at a distance of : 0.3660390362216514
[-3.08887716 -0.26091244], at a distance of : 0.37210137381041425
[ 2.44623445 -4.93592745], at a distance of : 0.4872902483369646


# Find the largest Square in an NxM matrix with obstructions

Given an NxM matrix, with zeros and ones (open spaces and obstructions), find the largest sub-matrix, that is square dxd (including 1x1), with no obstructions within it.

## Solution:

Using dynamic programming and memoization we can fill a memoization matrix, of the same size as the input, with previously computed solutions.  

The solution at x, y is the minimum of the previously computed solutions at: 

x+1, y  

x, y+1

x+1, y+1

## Complexity:

We have to touch every element of the input matrix once, so the run-time is O(n X m)

We have to keep a memoization matrix of the same size, so our space complexity is O(n X m)

In [6]:
# generate matrix for problem
from random import shuffle
def create_nxm_matrix_with_obstructions(n, m, n_obs):
    """ 
    n, m are size of matrix
    n_obs is the number of osbtructions
    """
    mat = np.zeros([n,m])
    indices = [(i,j) for i in range(n) for j in range(m)]
    shuffle(indices)
    for i,j in indices[:n_obs]:
        mat[i][j] = 1
        
    return mat


def find_largest_unobstructed_square(mat):
    n, m  = mat.shape
    mem_mat = np.ones([n,m]) * -1
    maxs = 0
    maxs_x = 0
    maxs_y = 0
    
    def solve(x, y):
        nonlocal n, m, mat, mem_mat, maxs, maxs_x, maxs_y

        if y > n - 1 or x > m - 1:
            return 0
        
        if not mem_mat[y][x] == -1:
            # we've computed this before, so use memo
            return mem_mat[y][x]
        
        # base case
        if y - 1 == n or y - 1 == m:
            # we hit the edge
            if mat[y][x] == 1:
                res = 0
            else:
                res = 1
                
        elif mat[y][x] == 0:
            # recurse three neighbors, take min of those solutions
            s1 = solve(x+1, y)
            s2 = solve(x, y+1)
            s3 = solve(x+1, y+1)
            res = min(s1, s2, s3)
            if res > 0:
                if y + res < n - 1 and x + res < m - 1:
                    res += 1
            else:
                # base case
                res = 1
        else:
            # recurse three neighbors
            solve(x+1, y)
            solve(x, y+1)
            solve(x+1, y+1)
            res = 0
        
        if res > maxs:
            maxs = res
            maxs_x = x
            maxs_y = y
        
        mem_mat[y][x] = res
        
        return res
    
    
    solve(0, 0)
    print("For input matrix:")
    print(mat)
    print("")
    print("Largest square has size of {0}, starting at x = {1} and y = {2}".format(maxs, maxs_x, maxs_y))
    print("")
    print("The Memo. matrix looks like: ")
    print(mem_mat)

In [7]:
n = 10
m = 8
n_obs = 8
mat = create_nxm_matrix_with_obstructions(n, m, n_obs)

find_largest_unobstructed_square(mat)

For input matrix:
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]]

Largest square has size of 4.0, starting at x = 1 and y = 1

The Memo. matrix looks like: 
[[ 2.  4.  3.  2.  2.  2.  1.  1.]
 [ 1.  4.  3.  2.  1.  1.  1.  1.]
 [ 0.  3.  3.  2.  1.  0.  0.  1.]
 [ 2.  2.  3.  3.  2.  2.  1.  1.]
 [ 1.  1.  2.  2.  2.  1.  1.  1.]
 [ 1.  0.  2.  1.  1.  1.  0.  1.]
 [ 3.  3.  2.  1.  0.  2.  1.  1.]
 [ 2.  2.  2.  2.  2.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  0.]]
